# Multi-Period Rebalancing Strategies

Analysis of dynamic portfolio strategies over time with rebalancing.

## Key Questions

1. How often should we rebalance?
2. Does Kelly criterion beat buy-and-hold?
3. What's the impact of transaction costs?
4. How do strategies perform in different market regimes?
5. Dynamic vs static allocation?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')

from portfolio_gambling.single_period import SinglePeriodOptimizer
from portfolio_gambling.multi_period import MultiPeriodOptimizer
from portfolio_gambling.utils import generate_returns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)

print("Multi-Period Rebalancing Strategies")
print("=" * 50)

## 1. Generate Market Data with Regime Changes

In [ ]:
# Generate multi-regime market data
n_assets = 4
n_periods_per_regime = 500
n_regimes = 3
n_total = n_periods_per_regime * n_regimes

# Define different market regimes
regimes = {
    'Bull Market': {
        'mean': np.array([0.12, 0.15, 0.10, 0.08]) / 252,
        'vol': np.array([0.15, 0.25, 0.18, 0.12]) / np.sqrt(252)
    },
    'Bear Market': {
        'mean': np.array([-0.10, -0.05, -0.08, 0.02]) / 252,
        'vol': np.array([0.30, 0.40, 0.35, 0.15]) / np.sqrt(252)
    },
    'Sideways': {
        'mean': np.array([0.05, 0.06, 0.04, 0.03]) / 252,
        'vol': np.array([0.20, 0.25, 0.22, 0.15]) / np.sqrt(252)
    }
}

# Correlation matrix (same across regimes)
corr = np.array([
    [1.0, 0.7, 0.5, 0.3],
    [0.7, 1.0, 0.6, 0.4],
    [0.5, 0.6, 1.0, 0.5],
    [0.3, 0.4, 0.5, 1.0]
])

# Generate returns for each regime
all_returns = []
regime_labels = []

for regime_name in ['Bull Market', 'Bear Market', 'Sideways']:
    params = regimes[regime_name]
    cov = np.outer(params['vol'], params['vol']) * corr
    regime_returns = generate_returns(n_assets, n_periods_per_regime, 
                                     params['mean'], cov)
    all_returns.append(regime_returns)
    regime_labels.extend([regime_name] * n_periods_per_regime)

returns = np.vstack(all_returns)

print(f"Generated {n_total} periods across {n_regimes} regimes")
print(f"  Assets: {n_assets}")
print(f"  Periods per regime: {n_periods_per_regime}")
print(f"\nRegimes: Bull Market → Bear Market → Sideways")

## 2. Define Rebalancing Strategies

In [ ]:
def simulate_rebalancing(returns, initial_weights, rebalance_freq=1, 
                        transaction_cost=0.0, strategy_type='static'):
    """
    Simulate portfolio with rebalancing.
    
    Parameters:
    -----------
    returns : array
        Return matrix
    initial_weights : array
        Starting weights
    rebalance_freq : int
        Rebalance every N periods (1 = daily, 21 = monthly, etc.)
    transaction_cost : float
        Cost per dollar traded (e.g., 0.001 = 10 bps)
    strategy_type : str
        'static', 'dynamic_kelly', 'buy_hold'
    """
    n_periods = len(returns)
    n_assets = returns.shape[1]
    
    wealth = np.zeros(n_periods + 1)
    wealth[0] = 1.0
    
    weights_history = []
    turnover_history = []
    
    current_weights = initial_weights.copy()
    
    for t in range(n_periods):
        # Record weights
        weights_history.append(current_weights.copy())
        
        # Apply returns
        port_return = current_weights @ returns[t]
        wealth[t+1] = wealth[t] * (1 + port_return)
        
        # Weights drift due to returns
        asset_values = current_weights * (1 + returns[t])
        drifted_weights = asset_values / np.sum(asset_values)
        
        # Rebalance?
        if (t + 1) % rebalance_freq == 0:
            if strategy_type == 'static':
                target_weights = initial_weights
            elif strategy_type == 'dynamic_kelly':
                # Recompute Kelly based on recent history
                window = min(t+1, 252)  # Use last year of data
                recent_returns = returns[max(0, t-window+1):t+1]
                opt = MultiPeriodOptimizer(recent_returns)
                target_weights = opt.kelly_criterion()
            elif strategy_type == 'buy_hold':
                target_weights = drifted_weights  # No rebalancing
            
            # Calculate turnover
            turnover = np.sum(np.abs(target_weights - drifted_weights))
            turnover_history.append(turnover)
            
            # Apply transaction costs
            cost = turnover * transaction_cost * wealth[t+1]
            wealth[t+1] -= cost
            
            current_weights = target_weights
        else:
            current_weights = drifted_weights
            turnover_history.append(0)
    
    return {
        'wealth': wealth,
        'weights_history': np.array(weights_history),
        'turnover': np.array(turnover_history),
        'final_wealth': wealth[-1],
        'total_return': wealth[-1] - 1,
        'cagr': (wealth[-1]) ** (252/n_periods) - 1,
        'total_turnover': np.sum(turnover_history)
    }

print("Defined rebalancing simulator")

## 3. Compute Initial Portfolios

In [ ]:
# Use first regime to compute initial portfolios
init_returns = returns[:n_periods_per_regime]

sp_opt = SinglePeriodOptimizer(init_returns)
mp_opt = MultiPeriodOptimizer(init_returns)

portfolios = {
    'Kelly': mp_opt.kelly_criterion(),
    'Half Kelly': mp_opt.fractional_kelly(0.5),
    'Mean-Variance': sp_opt.mean_variance(),
    'Equal Weight': np.ones(n_assets) / n_assets,
    'Risk Parity': np.array([0.4, 0.2, 0.25, 0.15])  # Simplified
}

print("Initial Portfolio Weights:")
for name, weights in portfolios.items():
    print(f"  {name:20s}: {weights.round(3)}")

## 4. Compare Rebalancing Frequencies

In [ ]:
# Test different rebalancing frequencies for Kelly
rebal_freqs = {
    'Daily': 1,
    'Weekly': 5,
    'Monthly': 21,
    'Quarterly': 63,
    'Annual': 252,
    'Buy & Hold': n_total + 1  # Never rebalance
}

kelly_weights = portfolios['Kelly']
freq_results = {}

for name, freq in rebal_freqs.items():
    freq_results[name] = simulate_rebalancing(
        returns, kelly_weights, rebalance_freq=freq, 
        transaction_cost=0.001  # 10 bps
    )

# Display results
print("\nKelly Portfolio: Rebalancing Frequency Comparison")
print("=" * 70)
print(f"{'Frequency':15s} {'Final Wealth':>12s} {'CAGR':>8s} {'Total Turnover':>15s}")
print("=" * 70)

for name, res in freq_results.items():
    print(f"{name:15s} ${res['final_wealth']:11.2f} {res['cagr']*100:7.2f}% {res['total_turnover']:14.1f}")

## 5. Visualize Wealth Evolution

In [ ]:
# Plot wealth paths for different rebalancing frequencies
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Wealth paths
ax = axes[0]
for name, res in freq_results.items():
    ax.plot(res['wealth'], linewidth=2.5, label=name, alpha=0.8)

# Mark regime changes
for i, regime_start in enumerate([0, n_periods_per_regime, 2*n_periods_per_regime]):
    ax.axvline(x=regime_start, color='gray', linestyle='--', alpha=0.5)
    if i == 0:
        ax.text(regime_start + 50, ax.get_ylim()[1]*0.9, 'Bull', fontsize=10)
    elif i == 1:
        ax.text(regime_start + 50, ax.get_ylim()[1]*0.9, 'Bear', fontsize=10)
    else:
        ax.text(regime_start + 50, ax.get_ylim()[1]*0.9, 'Sideways', fontsize=10)

ax.set_title('Kelly Portfolio: Wealth Evolution by Rebalancing Frequency', 
            fontsize=14, fontweight='bold')
ax.set_xlabel('Time Period', fontsize=12)
ax.set_ylabel('Wealth ($)', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Log scale
ax2 = axes[1]
for name, res in freq_results.items():
    ax2.semilogy(res['wealth'], linewidth=2.5, label=name, alpha=0.8)

for regime_start in [0, n_periods_per_regime, 2*n_periods_per_regime]:
    ax2.axvline(x=regime_start, color='gray', linestyle='--', alpha=0.5)

ax2.set_title('Kelly Portfolio: Wealth Evolution (Log Scale)', 
             fontsize=14, fontweight='bold')
ax2.set_xlabel('Time Period', fontsize=12)
ax2.set_ylabel('Wealth ($, log scale)', fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- More frequent rebalancing = staying closer to target")
print("- But: more turnover = more transaction costs")
print("- Sweet spot often monthly/quarterly")

## 6. Compare Different Strategies

In [ ]:
# Run all strategies with monthly rebalancing
strategy_results = {}
rebal_freq = 21  # Monthly
transaction_cost = 0.001  # 10 bps

for name, weights in portfolios.items():
    strategy_results[name] = simulate_rebalancing(
        returns, weights, rebalance_freq=rebal_freq,
        transaction_cost=transaction_cost
    )

# Also add dynamic Kelly
strategy_results['Dynamic Kelly'] = simulate_rebalancing(
    returns, portfolios['Kelly'], rebalance_freq=rebal_freq,
    transaction_cost=transaction_cost, strategy_type='dynamic_kelly'
)

# Display comparison
print("\nStrategy Comparison (Monthly Rebalancing, 10 bps costs):")
print("=" * 70)
print(f"{'Strategy':20s} {'Final Wealth':>12s} {'CAGR':>8s} {'Turnover':>10s}")
print("=" * 70)

for name, res in strategy_results.items():
    print(f"{name:20s} ${res['final_wealth']:11.2f} {res['cagr']*100:7.2f}% {res['total_turnover']:9.1f}")

## 7. Visualize Strategy Performance

In [ ]:
# Create comprehensive comparison plot
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Wealth paths
ax = axes[0, 0]
for name, res in strategy_results.items():
    ax.plot(res['wealth'], linewidth=2.5, label=name, alpha=0.8)
for regime_start in [n_periods_per_regime, 2*n_periods_per_regime]:
    ax.axvline(x=regime_start, color='gray', linestyle='--', alpha=0.3)
ax.set_title('Wealth Evolution', fontsize=14, fontweight='bold')
ax.set_xlabel('Time')
ax.set_ylabel('Wealth ($)')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Drawdowns
ax = axes[0, 1]
for name, res in strategy_results.items():
    wealth = res['wealth']
    running_max = np.maximum.accumulate(wealth)
    drawdown = (wealth - running_max) / running_max
    ax.plot(drawdown * 100, linewidth=2, label=name, alpha=0.8)
ax.set_title('Drawdowns', fontsize=14, fontweight='bold')
ax.set_xlabel('Time')
ax.set_ylabel('Drawdown (%)')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Final wealth distribution (bar chart)
ax = axes[1, 0]
names = list(strategy_results.keys())
final_wealths = [strategy_results[name]['final_wealth'] for name in names]
ax.bar(range(len(names)), final_wealths, color='steelblue', edgecolor='black', linewidth=1.5)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right')
ax.set_title('Final Wealth Comparison', fontsize=14, fontweight='bold')
ax.set_ylabel('Final Wealth ($)')
ax.grid(True, alpha=0.3, axis='y')

# Turnover comparison
ax = axes[1, 1]
turnovers = [strategy_results[name]['total_turnover'] for name in names]
ax.bar(range(len(names)), turnovers, color='coral', edgecolor='black', linewidth=1.5)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right')
ax.set_title('Total Turnover', fontsize=14, fontweight='bold')
ax.set_ylabel('Turnover')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 8. Impact of Transaction Costs

In [ ]:
# Test different transaction cost levels
cost_levels = [0, 0.0005, 0.001, 0.002, 0.005]  # 0 to 50 bps
kelly_w = portfolios['Kelly']

cost_impact = {}
for cost in cost_levels:
    res = simulate_rebalancing(returns, kelly_w, rebalance_freq=21, 
                              transaction_cost=cost)
    cost_impact[cost] = res

# Plot
plt.figure(figsize=(14, 7))
for cost, res in cost_impact.items():
    label = f"{cost*10000:.0f} bps" if cost > 0 else "No costs"
    plt.plot(res['wealth'], linewidth=2.5, label=label)

plt.title('Kelly Portfolio: Impact of Transaction Costs', fontsize=14, fontweight='bold')
plt.xlabel('Time Period', fontsize=12)
plt.ylabel('Wealth ($)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nTransaction Cost Impact on Final Wealth:")
print("=" * 50)
for cost, res in cost_impact.items():
    label = f"{cost*10000:.0f} bps"
    print(f"{label:10s}: ${res['final_wealth']:.2f} (CAGR: {res['cagr']*100:.2f}%)")

print("\nHigher costs → less frequent rebalancing is optimal")

## 9. Dynamic vs Static Allocation

In [ ]:
# Compare static Kelly vs dynamic Kelly
static_kelly = strategy_results['Kelly']
dynamic_kelly = strategy_results['Dynamic Kelly']

fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Wealth comparison
ax = axes[0]
ax.plot(static_kelly['wealth'], linewidth=3, label='Static Kelly', color='blue')
ax.plot(dynamic_kelly['wealth'], linewidth=3, label='Dynamic Kelly', color='red', linestyle='--')
for regime_start in [n_periods_per_regime, 2*n_periods_per_regime]:
    ax.axvline(x=regime_start, color='gray', linestyle='--', alpha=0.3)
ax.set_title('Static vs Dynamic Kelly', fontsize=14, fontweight='bold')
ax.set_xlabel('Time Period')
ax.set_ylabel('Wealth ($)')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# Weight evolution for dynamic Kelly
ax = axes[1]
weights_hist = dynamic_kelly['weights_history']
for i in range(n_assets):
    ax.plot(weights_hist[:, i], linewidth=2, label=f'Asset {i+1}', alpha=0.7)
for regime_start in [n_periods_per_regime, 2*n_periods_per_regime]:
    ax.axvline(x=regime_start, color='gray', linestyle='--', alpha=0.3)
ax.set_title('Dynamic Kelly: Weight Evolution', fontsize=14, fontweight='bold')
ax.set_xlabel('Time Period')
ax.set_ylabel('Portfolio Weight')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDynamic Kelly adapts to changing market conditions")
print("But: more turnover and potential overfitting to recent data")

## Key Takeaways

### Rebalancing Frequency:
1. **Too frequent**: High transaction costs eat returns
2. **Too infrequent**: Drift from targets, suboptimal allocations
3. **Sweet spot**: Monthly to quarterly for most strategies
4. **Depends on**: Transaction costs, portfolio turnover, market volatility

### Transaction Costs Matter:
- 10-20 bps per trade is realistic for retail
- Can significantly impact long-term wealth
- Higher costs → less frequent rebalancing
- Consider bid-ask spreads, market impact

### Dynamic vs Static:
- **Dynamic**: Adapts to regimes, potentially higher returns
- **Static**: Simpler, lower turnover, less overfitting
- **Hybrid**: Rebalance to fixed targets, but adjust targets annually

### Kelly Criterion:
- Works well with regular rebalancing
- Buy-and-hold Kelly drifts away from optimal
- Fractional Kelly reduces turnover need

### Practical Recommendations:
1. **Retail investors**: Quarterly rebalancing
2. **Institutional**: Monthly with thresholds (e.g., ±5% drift)
3. **High turnover strategies**: Factor in costs explicitly
4. **Consider**: Tax implications (wash sales, capital gains)
5. **Monitor**: Actual vs target weights, trigger rebalance on drift